In [2]:
import pandas as pd

from StateMachines.TradingStateMachine import TradingStateMachine
from StrategyBuilders import getStrategyBuilder
from Strategies.AbstractTradingStrategy import AbstractTradingStrategy

In [3]:
folder = "./Data/bybit"
folder_lab1 = "./Data/lab1"
file = "ETH-USDT:USDT_1m.csv"
periods = [5, 10, 15, 20, 30, 60, 120]

In [4]:
folder = "./Data/gemini"
folder_lab1 = "./Data/lab1"
file = "gemini_1m_features.csv"


In [5]:

strategy_builder = getStrategyBuilder('SIMPLE')
trader: TradingStateMachine = TradingStateMachine(strategy_builder("gemini_btcusd_1m"))

df = pd.read_csv(f"{folder}/{file}")
len(df)

3680586

In [6]:
period = 30
periods = [period]
first_year = 2017

In [7]:
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['year_'] = df['timestamp'].dt.year

features_base = ['timestamp', 'close', 'year_']
columns = features_base + AbstractTradingStrategy.get_features_list(df)
features_stochrsi = [col for col in columns if 'stoch_rsi' in col and 'change_of_trend' not in col and col[-1] == '_']
features_hma = [col for col in columns if 'hma' in col and 'change_of_trend' not in col and col[-1] == '_']
features_macd = [col for col in columns if 'macd' in col and 'change_of_trend' not in col and col[-1] == '_']


In [39]:
def get_best_group(df, features, period, sort='mean', max=5, ascending=False, head=True):
    
    column = f'close_p_in_{period}'
    grouped_df = df.groupby(features)[column].agg(['count', 'mean', 'min', 'max'])
    grouped_df = grouped_df.sort_values(by=sort, ascending=ascending)
    if head:
        return grouped_df.head(max)
    else:
        return grouped_df.tail(max)

def convert_grouped_to_dataframe(grouped_df):
    df = grouped_df.reset_index()
    return df

# import pandas as pd

def append_dataframes(df1, df2):
    # Concatenate the dataframes
    df = pd.concat([df1, df2], ignore_index=True)
    return df

we want to work on the history 1m from bitcoin (gemini)

Dataframe must be indexed by timestamp

In [9]:
df = df[columns]
df = df.set_index('timestamp')

df.tail(5)

,close,year_,stoch_rsi_overbought_,stoch_rsi_oversold_,stoch_rsi_bullish_crossover_,stoch_rsi_bearish_crossover_,stoch_rsi_trend_,stoch_rsi_change_of_trend_,hma200_trend_,hma200_change_of_trend_,hma200_above_price_,macd_bullish_crossover_,macd_bearish_crossover_,macd_positive_,macd_trend_,macd_change_of_trend_,year_
timestamp,,,,,,,,,,,,,,,,,
2024-04-07 00:24:00,69046.77,2024,False,False,False,False,Downtrend,NaN,Downtrend,NaN,True,False,False,True,Uptrend,NaN,2024
2024-04-07 00:25:00,68944.63,2024,False,True,False,False,Downtrend,NaN,Downtrend,NaN,True,False,True,False,Uptrend,NaN,2024
2024-04-07 00:26:00,68942.84,2024,False,True,False,False,Downtrend,NaN,Downtrend,NaN,True,False,False,False,Downtrend,Uptrend->Downtrend,2024
2024-04-07 00:27:00,68947.39,2024,False,True,False,False,Downtrend,NaN,Downtrend,NaN,True,False,False,False,Downtrend,NaN,2024
2024-04-07 00:28:00,68890.02,2024,False,True,True,False,Downtrend,NaN,Downtrend,NaN,True,False,False,False,Downtrend,NaN,2024


Now we calculate the result for different number of intervals

In [10]:
# adding shifted periods results
features_close = [f'close_in_{period}' for period in periods]
features_close_perc = [f'close_p_in_{period}' for period in periods]
for period in periods:
    column_name = f'close_in_{period}'
    df[column_name] = df['close'].shift(-period)
    column_perc = f'close_p_in_{period}'
    df[column_perc] = ((df['close'].shift(-period) - df['close']) / df['close']) * 100
    
df.to_csv(f"{folder_lab1}/{file}_periods.csv")
df.columns

Index(['close', 'year_', 'stoch_rsi_overbought_', 'stoch_rsi_oversold_',
       'stoch_rsi_bullish_crossover_', 'stoch_rsi_bearish_crossover_',
       'stoch_rsi_trend_', 'stoch_rsi_change_of_trend_', 'hma200_trend_',
       'hma200_change_of_trend_', 'hma200_above_price_',
       'macd_bullish_crossover_', 'macd_bearish_crossover_', 'macd_positive_',
       'macd_trend_', 'macd_change_of_trend_', 'year_', 'close_in_30',
       'close_p_in_30'],
      dtype='object')

we classify the features based on the result for the given number of intervals

In [11]:
# period = 30
x = df[df[f'close_p_in_{period}'] > 1]
x[['year_', 'close', f'close_in_{period}', f'close_p_in_{period}']].head(10)

,year_,year_,close,close_in_30,close_p_in_30
timestamp,,,,,
2017-01-01 16:11:00,2017,2017,982.99,994.95,1.216696
2017-01-01 16:12:00,2017,2017,982.99,996.32,1.356067
2017-01-01 16:13:00,2017,2017,982.99,996.32,1.356067
2017-01-01 16:14:00,2017,2017,982.99,1000.01,1.731452
2017-01-01 16:15:00,2017,2017,982.99,1000.00,1.730435
2017-01-01 16:16:00,2017,2017,982.97,999.99,1.731487
2017-01-01 16:17:00,2017,2017,982.51,996.71,1.445278
2017-01-01 16:18:00,2017,2017,979.34,998.70,1.976842
2017-01-01 16:19:00,2017,2017,979.34,998.70,1.976842


In [12]:
# Create a column variable that is the combination of all the features_ lists
all_features = features_stochrsi + features_hma + features_macd

# removing likely not valid rows
features = all_features + [f'close_p_in_{period}']
new_df = df[features].iloc[200:-period]

len(new_df)


3680356

We reduce the dataframe to the last 4 years

In [13]:

new_df = new_df[new_df.index.year >= first_year]
len(new_df)

3680356

In [14]:
# import pandas as pd

# # Assuming `df` is your DataFrame with OHLC data, features, and a 'future_return' column 
# # that represents the percent increase after N periods.

# # Calculate Pearson correlation coefficients
# correlation_matrix = new_df.corr()
# performance_correlations = correlation_matrix[f'close_p_in_{period}'].sort_values(ascending=False)

# print(performance_correlations)

1. Chi-Square Test for Independence
The Chi-Square test is useful for testing the relationship between two categorical variables. It can tell you whether the distribution of sample categorical data matches an expected distribution.

Use Case: To determine if there is a significant association between two categorical features.

In [30]:
import numpy as np
from scipy.stats import chi2_contingency

# Assuming df is your DataFrame and 'feature' and 'target' are your categorical columns
def chi2_test(df, feature, target):
    contingency_table = pd.crosstab(df[feature], df[target])
    chi2, p, dof, expected = chi2_contingency(contingency_table)
    return chi2, p

def chi2_feature_selection(df, features, target):
    results = []
    for feature in features:
        chi2, p = chi2_test(df, feature, target)
        results.append({'feature': feature, 'chi2': chi2, 'p-value': p})
    return results

print(new_df.columns)





Index(['stoch_rsi_overbought_', 'stoch_rsi_oversold_',
       'stoch_rsi_bullish_crossover_', 'stoch_rsi_bearish_crossover_',
       'stoch_rsi_trend_', 'hma200_trend_', 'hma200_above_price_',
       'macd_bullish_crossover_', 'macd_bearish_crossover_', 'macd_positive_',
       'macd_trend_', 'close_p_in_30', 'target'],
      dtype='object')


Selecting the best features for BUY signal

In [33]:
print(new_df.columns)
results = []
new_df['target'] = np.where(new_df[f'close_p_in_{period}'] > 0, 'Increase', 'Decrease')
for feature in all_features:
    contingency_table = pd.crosstab(new_df[feature], new_df['target'])
    chi2, p, dof, expected = chi2_contingency(contingency_table)
    results.append({'feature': feature, 'chi2': chi2, 'p-value': p})

# Sort the results by chi2 in descending order
results = sorted(results, key=lambda x: x['chi2'], reverse=True)

# Display the sorted results
for result in results:
    print(f"Feature: {result['feature']}, Chi-Square Value: {result['chi2']}, P-Value: {result['p-value']}")


Index(['stoch_rsi_overbought_', 'stoch_rsi_oversold_',
       'stoch_rsi_bullish_crossover_', 'stoch_rsi_bearish_crossover_',
       'stoch_rsi_trend_', 'hma200_trend_', 'hma200_above_price_',
       'macd_bullish_crossover_', 'macd_bearish_crossover_', 'macd_positive_',
       'macd_trend_', 'close_p_in_30', 'target'],
      dtype='object')
Feature: hma200_trend_, Chi-Square Value: 12341.981646970511, P-Value: 0.0
Feature: macd_trend_, Chi-Square Value: 6317.035989819508, P-Value: 0.0
Feature: hma200_above_price_, Chi-Square Value: 4403.817489622915, P-Value: 0.0
Feature: macd_positive_, Chi-Square Value: 935.2414216253367, P-Value: 2.142499522495479e-205
Feature: stoch_rsi_oversold_, Chi-Square Value: 591.6792414129565, P-Value: 1.0805656720444698e-130
Feature: stoch_rsi_overbought_, Chi-Square Value: 309.20155481951434, P-Value: 3.259577502263205e-69
Feature: stoch_rsi_trend_, Chi-Square Value: 18.357771462497553, P-Value: 0.00010319545658255706
Feature: macd_bearish_crossover_, Chi

In [34]:
cluster1 = ['hma200_trend_','hma200_above_price_','macd_trend_']
best_df = get_best_group(new_df, cluster1, period)

save_model = False
if save_model:
    print("saving model")
    x = convert_grouped_to_dataframe(best_df)
    x = x[cluster1 ]
    x['signal'] = 'Buy'
    x.to_csv(f"./simple_model_buy.csv")

best_df

count      mean        min  \
hma200_trend_ hma200_above_price_ macd_trend_                                 
Not trending  False               Uptrend          255  0.027985  -1.556668   
              True                Not trending     961  0.018049  -3.434768   
Downtrend     True                Downtrend     633405  0.014052 -38.867640   
Not trending  False               Downtrend        131  0.013730  -2.950425   
Uptrend       False               Downtrend     631711  0.011664 -17.231202   

                                                      max  
hma200_trend_ hma200_above_price_ macd_trend_              
Not trending  False               Uptrend        1.900347  
              True                Not trending   3.211990  
Downtrend     True                Downtrend     28.299313  
Not trending  False               Downtrend      1.583724  
Uptrend       False               Downtrend     16.114799

In [36]:
print(new_df.columns)
results = []
new_df['target'] = np.where(new_df[f'close_p_in_{period}'] < 0, 'Decrease', 'Increase')
for feature in all_features:
    contingency_table = pd.crosstab(new_df[feature], new_df['target'])
    chi2, p, dof, expected = chi2_contingency(contingency_table)
    results.append({'feature': feature, 'chi2': chi2, 'p-value': p})

# Sort the results by chi2 in descending order
results = sorted(results, key=lambda x: x['chi2'], reverse=True)

# Display the sorted results
for result in results:
    print(f"Feature: {result['feature']}, Chi-Square Value: {result['chi2']}, P-Value: {result['p-value']}")

Index(['stoch_rsi_overbought_', 'stoch_rsi_oversold_',
       'stoch_rsi_bullish_crossover_', 'stoch_rsi_bearish_crossover_',
       'stoch_rsi_trend_', 'hma200_trend_', 'hma200_above_price_',
       'macd_bullish_crossover_', 'macd_bearish_crossover_', 'macd_positive_',
       'macd_trend_', 'close_p_in_30', 'target'],
      dtype='object')
Feature: hma200_trend_, Chi-Square Value: 11716.319870695103, P-Value: 0.0
Feature: macd_trend_, Chi-Square Value: 5590.914506561879, P-Value: 0.0
Feature: hma200_above_price_, Chi-Square Value: 4055.1744286278426, P-Value: 0.0
Feature: stoch_rsi_trend_, Chi-Square Value: 1257.830660889887, P-Value: 7.337403514496675e-274
Feature: macd_positive_, Chi-Square Value: 996.9316665191881, P-Value: 8.34088281992188e-219
Feature: stoch_rsi_overbought_, Chi-Square Value: 440.07479884159477, P-Value: 1.042718575551122e-97
Feature: stoch_rsi_oversold_, Chi-Square Value: 384.3778024898904, P-Value: 1.3863057048926612e-85
Feature: stoch_rsi_bearish_crossover_, 

In [41]:
cluster1 = ['hma200_trend_','hma200_above_price_','macd_trend_']
best_df = get_best_group(new_df, cluster1, period, ascending=False, head=False)

save_model = True
if save_model:
    print("saving model")
    x = convert_grouped_to_dataframe(best_df)
    x = x[cluster1 ]
    x['signal'] = 'Sell'
    x.to_csv(f"./simple_model_sell.csv")

best_df

saving model


count      mean       min  \
hma200_trend_ hma200_above_price_ macd_trend_                               
Downtrend     True                Not trending   3907 -0.013276 -4.064648   
Uptrend       False               Not trending   3051 -0.014244 -2.578498   
Not trending  True                Downtrend       267 -0.014494 -1.796370   
              False               Not trending   3652 -0.058708 -6.602463   
Uptrend       True                Not trending    760 -0.115130 -2.646651   

                                                     max  
hma200_trend_ hma200_above_price_ macd_trend_             
Downtrend     True                Not trending  1.501541  
Uptrend       False               Not trending  2.672373  
Not trending  True                Downtrend     1.230483  
              False               Not trending  3.898413  
Uptrend       True                Not trending  1.088616

In [18]:
column = f'close_p_in_{period}'

# Aggregations for count, min, and max
aggregations = {
    'count': (column, 'count'),
    'min': (column, 'min'),
    'max': (column, 'max')
}

# Perform the aggregations
result = df.agg(**aggregations)

# Count values greater than 0
greater_than_zero_count = (df[column] > 0).sum()

# Count values less than 0
less_than_zero_count = (df[column] < 0).sum()

# Adding these counts to the result
result['greater_than_zero'] = greater_than_zero_count
result['less_than_zero'] = less_than_zero_count

result


,close_p_in_30,greater_than_zero,less_than_zero
count,3.680556e+06,1859027,1778840
min,-3.886764e+01,1859027,1778840
max,3.530435e+01,1859027,1778840


In [19]:

get_best_group(new_df, features_stochrsi, period)

count  \
stoch_rsi_overbought_ stoch_rsi_oversold_ stoch_rsi_bullish_crossover_ stoch_rsi_bearish_crossover_ stoch_rsi_trend_          
False                 False               True                         False                        Not trending        189   
True                  False               False                        False                        Not trending      56539   
                                          True                         False                        Not trending        681   
                                          False                        True                         Not trending       2101   
False                 True                True                         False                        Uptrend           46022   

                                                                                                                          mean  \
stoch_rsi_overbought_ stoch_rsi_oversold_ stoch_rsi_bullish_crossover_ stoch_rsi_bearish_crossover_ stoch_rsi_trend_             
False                 False               True                         False                        Not trending      0.085468   
True                  False               False                        False                        Not trending      0.054284   
                                          True                         False                        Not trending      0.026956   
                                          False                        True                         Not trending      0.025256   
False                 True                True                         False                        Uptrend           0.013117   

                                                                                                                            min  \
stoch_rsi_overbought_ stoch_rsi_oversold_ stoch_rsi_bullish_crossover_ stoch_rsi_bearish_crossover_ stoch_rsi_trend_              
False                 False               True                         False                        Not trending      -2.182746   
True                  False               False                        False                        Not trending      -9.628426   
                                          True                         False                        Not trending      -5.158465   
                                          False                        True                         Not trending      -6.834781   
False                 True                True                         False                        Uptrend          -38.732548   

                                                                                                                            max  
stoch_rsi_overbought_ stoch_rsi_oversold_ stoch_rsi_bullish_crossover_ stoch_rsi_bearish_crossover_ stoch_rsi_trend_             
False                 False               True                         False                        Not trending       6.968734  
True                  False               False                        False                        Not trending       8.970100  
                                          True                         False                        Not trending       4.827530  
                                          False                        True                         Not trending       5.143171  
False                 True                True                         False                        Uptrend           12.385252

In [20]:

df_result = get_best_group(new_df, all_features, period, max=10, sort='mean')
df_result

count  \
stoch_rsi_overbought_ stoch_rsi_oversold_ stoch_rsi_bullish_crossover_ stoch_rsi_bearish_crossover_ stoch_rsi_trend_ hma200_trend_ hma200_above_price_ macd_bullish_crossover_ macd_bearish_crossover_ macd_positive_ macd_trend_           
False                 True                False                        True                         Uptrend          Uptrend       False               True                    False                   True           Downtrend         2   
                                          True                         False                        Not trending     Not trending  False               False                   False                   False          Not trending      1   
True                  False               False                        False                        Not trending     Not trending  False               False                   False                   False          Not trending     28   
                                                                                                    Uptrend          Not trending  False               True                    False                   True           Downtrend         1   
False                 True                True                         False                        Uptrend          Not trending  False               False                   False                   False          Not trending      5   
                      False               False                        False                        Downtrend        Not trending  True                False                   True                    False          Uptrend           2   
True                  False               True                         False                        Not trending     Downtrend     False               False                   False                   False          Downtrend         1   
                                          False                        True                         Downtrend        Uptrend       False               True                    False                   True           Uptrend           1   
False                 False               True                         False                        Downtrend        Not trending  False               False                   True                    False          Uptrend           1   
                                                                                                    Uptrend          Uptrend       False               False                   False                   False          Not trending      6   

                                                                                                                                                                                                                                        mean  \
stoch_rsi_overbought_ stoch_rsi_oversold_ stoch_rsi_bullish_crossover_ stoch_rsi_bearish_crossover_ stoch_rsi_trend_ hma200_trend_ hma200_above_price_ macd_bullish_crossover_ macd_bearish_crossover_ macd_positive_ macd_trend_              
False                 True                False                        True                         Uptrend          Uptrend       False               True                    False                   True           Downtrend     1.193243   
                                          True                         False                        Not trending     Not trending  False               False                   False                   False          Not trending  1.079951   
True                  False               False                        False                        Not trending     Not trending  False               False                   False                   False          Not trending  0.785207   
                                                                                                    Uptrend          Not trending  False               True                    False   

In [21]:

get_best_group(new_df, features_macd, period)

count  \
macd_bullish_crossover_ macd_bearish_crossover_ macd_positive_ macd_trend_             
False                   True                    False          Not trending      249   
                        False                   False          Downtrend     1616852   
True                    False                   True           Downtrend      102042   
False                   True                    False          Downtrend       36203   
                        False                   True           Downtrend       95154   

                                                                                 mean  \
macd_bullish_crossover_ macd_bearish_crossover_ macd_positive_ macd_trend_              
False                   True                    False          Not trending  0.010716   
                        False                   False          Downtrend     0.009558   
True                    False                   True           Downtrend     0.007519   
False                   True                    False          Downtrend     0.007215   
                        False                   True           Downtrend     0.006953   

                                                                                   min  \
macd_bullish_crossover_ macd_bearish_crossover_ macd_positive_ macd_trend_               
False                   True                    False          Not trending  -1.762366   
                        False                   False          Downtrend    -38.812110   
True                    False                   True           Downtrend    -38.867640   
False                   True                    False          Downtrend    -10.844406   
                        False                   True           Downtrend    -17.231202   

                                                                                   max  
macd_bullish_crossover_ macd_bearish_crossover_ macd_positive_ macd_trend_              
False                   True                    False          Not trending   2.398996  
                        False                   False          Downtrend     22.190704  
True                    False                   True           Downtrend     28.299313  
False                   True                    False          Downtrend     14.257855  
                        False                   True           Downtrend     21.890070

In [22]:

get_best_group(new_df, features_stochrsi+features_macd, period)

count  \
stoch_rsi_overbought_ stoch_rsi_oversold_ stoch_rsi_bullish_crossover_ stoch_rsi_bearish_crossover_ stoch_rsi_trend_ macd_bullish_crossover_ macd_bearish_crossover_ macd_positive_ macd_trend_           
True                  False               False                        True                         Downtrend        True                    False                   True           Uptrend           2   
False                 True                True                         False                        Not trending     False                   False                   True           Downtrend         3   
                                                                                                                                                                     False          Not trending      4   
                                          False                        True                         Not trending     True                    False                   True           Uptrend           1   
                                          True                         False                        Downtrend        False                   True                    False          Not trending      1   

                                                                                                                                                                                                      mean  \
stoch_rsi_overbought_ stoch_rsi_oversold_ stoch_rsi_bullish_crossover_ stoch_rsi_bearish_crossover_ stoch_rsi_trend_ macd_bullish_crossover_ macd_bearish_crossover_ macd_positive_ macd_trend_              
True                  False               False                        True                         Downtrend        True                    False                   True           Uptrend       0.479425   
False                 True                True                         False                        Not trending     False                   False                   True           Downtrend     0.434968   
                                                                                                                                                                     False          Not trending  0.356938   
                                          False                        True                         Not trending     True                    False                   True           Uptrend       0.349005   
                                          True                         False                        Downtrend        False                   True                    False          Not trending  0.314345   

                                                                                                                                                                                                       min  \
stoch_rsi_overbought_ stoch_rsi_oversold_ stoch_rsi_bullish_crossover_ stoch_rsi_bearish_crossover_ stoch_rsi_trend_ macd_bullish_crossover_ macd_bearish_crossover_ macd_positive_ macd_trend_              
True                  False               False                        True                         Downtrend        True                    False                   True           Uptrend       0.303940   
False                 True                True                         False                        Not trending     False                   False                   True           Downtrend     0.111293   
                                                                                                                                                                     False          Not trending -0.087663   
                                          False                        True                         Not trending     True                    False                   True           Uptrend       0.349005   
                                          True                         False                 

In [23]:
folder_lab1 = "./Data/lab1"
features_to_test = features_stochrsi
for period in periods:
    print(f"Period: {period}")
    features = all_features + [f'close_p_in_{period}']
    new_df = df[features].iloc[50:-period]
    result = get_best_group(new_df, features_to_test, period, 'count')
    result.to_csv(f"{folder_lab1}/{file}_stochrsi_{period}.csv")
    print(result)
    
    # print(get_best_group(new_df, features_hma, period))
    # print(get_best_group(new_df, features_macd, period, 'count'))
    # print(get_best_group(new_df, features_stochrsi+features_macd, period))
    print("")

Period: 30
                                                                                                                       count  \
stoch_rsi_overbought_ stoch_rsi_oversold_ stoch_rsi_bullish_crossover_ stoch_rsi_bearish_crossover_ stoch_rsi_trend_           
True                  False               False                        False                        Uptrend           692148   
False                 True                False                        False                        Downtrend         675265   
                      False               False                        False                        Downtrend         570458   
                                                                                                    Uptrend           532496   
True                  False               False                        False                        Downtrend         173534   

                                                                                            

i want to find the best combination of features, maximising the results

now we find the combo of features that are performing better

In [24]:
import itertools

folder_lab1 = "./Data/lab1"
file = "gemini_1m_features.csv"
# features_to_test = features_stochrsi 
features_to_test = features_stochrsi 

combinations_stochrsi = []
for r in range(1, len(features_stochrsi) + 1):
    combinations_stochrsi.extend(itertools.combinations(features_stochrsi, r))
print(len(combinations_stochrsi)) 

combinations_hma = []
for r in range(1, len(features_hma) + 1):
    combinations_hma.extend(itertools.combinations(features_hma, r))
print(len(combinations_hma)) 

combinations_macd = []
for r in range(1, len(features_macd) + 1):
    combinations_macd.extend(itertools.combinations(features_macd, r))
print(len(combinations_macd)) 

cartesian_product = list(itertools.product(combinations_stochrsi, combinations_hma, combinations_macd))
print(len(cartesian_product))


31
3
15
1395


In [25]:

folder_lab1 = "./Data/lab1"
file = "gemini_1m_features.csv"

features_to_test = features_stochrsi 



for period in periods:
    analysis_df = pd.DataFrame()
    print(f"Period: {period}")
    features = all_features + [f'close_p_in_{period}']
    new_df = df[features].iloc[50:-period]
    for combination in combinations_stochrsi:
        list_of_combination = list(combination)
        if len(list_of_combination) < 3:
            continue
        result = get_best_group(new_df, list_of_combination, period, 'count')
        tmp_df = convert_grouped_to_dataframe(result)
        tmp_df['period'] = period
        analysis_df = append_dataframes(tmp_df,analysis_df)
        # print(result)
        print("")
    analysis_df.to_csv(f"{folder_lab1}/{file}_stochrsi_analysis_{period}.csv")

Period: 30


















In [26]:

analysis_df = pd.read_csv(f"{folder_lab1}/{file}_stochrsi_analysis_{period}.csv")

# Filter the dataframe to include only rows where the count is greater than 1
filtered_df = analysis_df[analysis_df['count'] > 1]

# I want to minimise the losses
# filtered_df = filtered_df[filtered_df['mean'] > filtered_df['min'].abs()]

# Sort the dataframe by mean in descending order and by the difference between min and max in ascending order
sorted_df = filtered_df.sort_values(by=['mean', 'max', 'min'], ascending=[False, True, True])

sorted_df.head(5)


,Unnamed: 0,stoch_rsi_overbought_,stoch_rsi_oversold_,stoch_rsi_bullish_crossover_,stoch_rsi_bearish_crossover_,stoch_rsi_trend_,count,mean,min,max,period
29,29,False,True,True,False,NaN,180565,0.011432,-38.732548,12.385252,30
49,49,NaN,True,True,False,NaN,180565,0.011432,-38.732548,12.385252,30
78,78,False,True,True,NaN,NaN,180565,0.011432,-38.732548,12.385252,30
44,44,NaN,True,True,NaN,Downtrend,132996,0.010983,-38.071665,11.322810,30
66,66,False,True,NaN,NaN,Downtrend,846836,0.010363,-38.812110,29.606152,30
